In [2]:
import requests
import json
import pandas as pd

- Get all shops

In [7]:
url = "https://www.foody.vn/__get/Place/HomeListPlace?page=1&lat=10.823099&lon=106.629664&count=12&districtId=&cateId=&cuisineId=&isReputation=&type=1"
all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử

requests_id = [1031704, 725868]

menu_df = pd.DataFrame(columns=['request_id','dish_type_id', 'dish_type_name'])
menu_dish_df = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
menu_dish_option_df = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
menu_dish_option_item_df = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

for request_id in requests_id:
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        
        data = delivery_dish_API.text
        dataJson = json.loads(data)
        menu_infos = dataJson['reply']['menu_infos']

        menu_df_temp = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_type_name'])
        menu_dish_df_temp = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
        menu_dish_option_df_temp = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
        menu_dish_option_item_df_temp = pd.DataFrame(columns=['request_id', 'dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

        for menu_info in menu_infos:
                menu_df_temp.loc[len(menu_df_temp.index)] = [request_id, menu_info['dish_type_id'], menu_info['dish_type_name']]
                for dish in menu_info['dishes']:
                        menu_dish_df_temp.loc[len(menu_dish_df_temp.index)] = [
                                request_id,
                                menu_info['dish_type_id'],
                                dish['id'],
                                dish['name'],
                                dish['description'],
                                dish['price']['value'],
                                dish['total_like'],
                                dish['is_available']
                        ]
                        for option in dish['options']:
                                menu_dish_option_df_temp.loc[len(menu_dish_option_df_temp.index)] = [
                                        request_id,
                                        menu_info['dish_type_id'],
                                        dish['id'],
                                        option['id'],
                                        option['option_items']['min_select'],
                                        option['option_items']['max_select']
                                ]
                                for item in option['option_items']['items']:
                                        menu_dish_option_item_df_temp.loc[len(menu_dish_option_item_df_temp.index)] = [
                                                request_id,
                                                menu_info['dish_type_id'],
                                                dish['id'],
                                                option['id'],
                                                item['id'],
                                                item['name'],
                                                item['price']['value']
                                        ]

        menu_df = pd.concat([menu_df, menu_df_temp])
        menu_dish_df = pd.concat([menu_dish_df, menu_dish_df_temp])
        menu_dish_option_df = pd.concat([menu_dish_option_df, menu_dish_option_df_temp])
        menu_dish_option_item_df = pd.concat([menu_dish_option_item_df, menu_dish_option_item_df_temp])
                                

- merge to a CSV

In [ ]:


menu = pd.merge(menu_dish_df, menu_df,  how='left', on=['request_id', 'dish_type_id'])
menu = pd.merge(menu, menu_dish_option_df,  how='left', left_on=['request_id', 'dish_type_id','dish_id'], right_on = ['request_id', 'dish_type_id','dish_id'])
menu = pd.merge(menu, menu_dish_option_item_df,  how='left', left_on=['request_id', 'dish_type_id','dish_id', 'option_id'],\
     right_on = ['request_id', 'dish_type_id','dish_id', 'option_id'])
menu.to_csv('menu.csv', sep=',', encoding='utf-8')



- get detail of ONE shop

In [6]:
# VD: có thể thay request_id = 1031704 để thử

requests_id = [1031704, 725868]

detail_df = pd.DataFrame(columns=['city_id', 'district_id', 'request_id', 'categories', 'cuisines', 'total_review', 'avg', \
                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'promotions', 'minimun_shiping_fee', 'is_foody_delivery', 'min_price', 'max_price'])

for request_id in requests_id:
        detail_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        
        data = detail_API.text
        dataJson = json.loads(data)
        detail_infos = dataJson['reply']['delivery_detail']

        detail_df_temp = pd.DataFrame(columns=['city_id', 'district_id', 'request_id', 'categories', 'cuisines', 'total_review', 'avg', \
                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'promotions', 'minimun_shiping_fee', 'is_foody_delivery', 'min_price', 'max_price'])

        
        detail_df_temp.loc[len(detail_df_temp.index)] = [detail_infos['city_id'], detail_infos['district_id'], request_id, detail_infos['categories'], \
                detail_infos['cuisines'], detail_infos['rating']['total_review'], detail_infos['rating']['avg'], detail_infos['delivery']['service_fee']['value'],\
                detail_infos['delivery']['avg_price']['value'], detail_infos['delivery']['min_order_value']['value'], detail_infos['delivery']['min_charge'], \
                detail_infos['delivery']['promotions'], detail_infos['delivery']['shipping_fee']['minimum_fee'], detail_infos['delivery']['is_foody_delivery'], \
                detail_infos['price_range']['min_price'], detail_infos['price_range']['max_price']]
        

        detail_df = pd.concat([detail_df, detail_df_temp])


detail_df.to_csv('detail_df.csv', sep=',', encoding='utf-8')

In [3]:
detail_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id=1031704&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        
data = detail_API.text
dataJson = json.loads(data)
detail_infos = dataJson['reply']['delivery_detail']

detail_infos

{'total_order': 0,
 'rating': {'total_review': 500,
  'avg': 4.7,
  'display_total_review': '500+',
  'app_link': 'foody://foody.vn?action=reviewlist&id=1031704'},
 'is_subscribe': False,
 'is_favorite': False,
 'city_id': 217,
 'phones': ['0776969606'],
 'restaurant_id': 1031704,
 'district_id': 16,
 'brand_id': 11607,
 'video': None,
 'asap_is_available': True,
 'contract_type': 1,
 'id': 121227,
 'location_url': 'ho-chi-minh',
 'foody_service_id': 1,
 'is_quality_merchant': True,
 'available_times': [{'date': '2022-09-29',
   'times': ['21:20',
    '21:25',
    '21:30',
    '21:35',
    '21:40',
    '21:45',
    '21:50',
    '21:55',
    '22:00',
    '22:05',
    '22:10',
    '22:15',
    '22:20',
    '22:25',
    '22:30',
    '22:35',
    '22:40',
    '22:45',
    '22:50',
    '22:55',
    '23:00',
    '23:05',
    '23:10',
    '23:15',
    '23:20',
    '23:25',
    '23:30',
    '23:35',
    '23:40',
    '23:45',
    '23:50',
    '23:55']},
  {'date': '2022-09-30',
   'times': ['09